In [ ]:
import os
import sys
import cv2
import time
import numpy as np
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from utilities import c2e




if __name__ == '__main__':

    
    # step2 trans cubemap to erp
    cube = np.zeros([6,3,2560,2560], dtype=np.float64)
    cubemap2erp = c2e(cubeW=2560, outH=2560, outW=2560*2,CUDA=True)

    if not os.path.exists('output/huawei_driving11/pinhole'):
        os.makedirs('output/huawei_driving11/pinhole')

    # get grid
    xc = (1920-1) / 2
    yc = (1080-1) / 2

    face_x, face_y = np.meshgrid(range(1920), range(1080))
    x = face_x - xc
    y = face_y - yc
    focal = xc / np.tan(np.radians(50))
    z = np.ones_like(x) * focal
    x,y,z = x/z, y/z, z/z
    theta = np.arctan2(x,z).astype(np.float64)
    phi = np.arctan2(y,np.sqrt(x*x+z*z)).astype(np.float64)
    theta = theta / np.pi
    phi = phi / np.pi * 2.0
    
    grid = np.stack([theta,phi],axis=-1)
    grid = np.expand_dims(grid, axis=0)
    grid = torch.from_numpy(grid)
    grid = grid.cuda()

    raw_data = np.load(f'output/huawei_driving11/cubemap/cm_rgb11_50.npz', allow_pickle=True)
    cv2.imwrite('left.png',raw_data['left_data'])
    cv2.imwrite('front.png',raw_data['front_data'])
    cv2.imwrite('right.png',raw_data['right_data'])

    for frame in range(2,100):
        raw_data = np.load(f'output/huawei_driving11/cubemap/cm_rgb11_{frame}.npz', allow_pickle=True)
        cube[0,:,:,:]=np.transpose(raw_data['back_data'],(2,0,1))
        cube[1,:,:,:]=np.transpose(raw_data['down_data'],(2,0,1))
        cube[2,:,:,:]=np.transpose(raw_data['front_data'],(2,0,1))
        cube[3,:,:,:]=np.transpose(raw_data['left_data'],(2,0,1))
        cube[4,:,:,:]=np.transpose(raw_data['right_data'],(2,0,1))
        cube[5,:,:,:]=np.transpose(raw_data['up_data'],(2,0,1))
        
        cube_tensor=torch.from_numpy(cube)
        out_erp = cubemap2erp.ToEquirecTensor(cube_tensor)
        out_pinhole = F.grid_sample(out_erp, grid, mode='bilinear', padding_mode='zeros', align_corners=True)

        out = out_pinhole.cpu().numpy()
        out = np.squeeze(out,axis=0)
        out = np.transpose(out,(1,2,0))
        out = out.astype(np.uint8)

        cv2.imwrite(os.path.join("output/huawei_driving11/pinhole", f'ph_rgb11_{frame}.jpg'), out, [int(cv2.IMWRITE_JPEG_QUALITY), 97])

In [ ]:
import numpy as np
import time
import threading
from queue import Queue

global_q = Queue(maxsize=10)


def save_data():
    i = 0
    while True:
        sim_cube_data = global_q.get()
        np.savez(
            f'test_save/{i}.npz',
            front   = sim_cube_data['front'],
            left    = sim_cube_data['left'],
            right   = sim_cube_data['right'],
            back    = sim_cube_data['back'],
            up      = sim_cube_data['up'],
            down    = sim_cube_data['down'],
        )
        i += 1

save_thread = threading.Thread(target=save_data)
save_thread.start()

start_time = time.time()
for i in range(100):
    sim_cube_data = {
        'front' : np.random.randint(0, 255, (2560,2560,3), dtype=np.uint8),
        'left'  : np.random.randint(0, 255, (2560,2560,3), dtype=np.uint8),
        'right' : np.random.randint(0, 255, (2560,2560,3), dtype=np.uint8),
        'back'  : np.random.randint(0, 255, (2560,2560,3), dtype=np.uint8),
        'up'    : np.random.randint(0, 255, (2560,2560,3), dtype=np.uint8),
        'down'  : np.random.randint(0, 255, (2560,2560,3), dtype=np.uint8),
    }
    
global_q.join()
use_time = time.time() - start_time
print(use_time)